## ДЗ к Уроку 6.
### Взаимосвязь величин. Параметрические и непараметрические показатели корреляции. Корреляционный анализ.

### 1. Задание:

_Даны значения величины заработной платы заемщиков банка (zp) и значения их поведенческого кредитного скоринга (ks):_

zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110],

ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832].

_Найдите ковариацию этих двух величин с помощью элементарных действий, а затем с помощью функции cov из numpy
Полученные значения должны быть равны.
Найдите коэффициент корреляции Пирсона с помощью ковариации и среднеквадратичных отклонений двух признаков,
а затем с использованием функций из библиотек numpy и pandas._

### 2. Задание:

_Измерены значения IQ выборки студентов,
обучающихся в местных технических вузах:_

131, 125, 115, 122, 131, 115, 107, 99, 125, 111.

_Известно, что в генеральной совокупности IQ распределен нормально.
Найдите доверительный интервал для математического ожидания с надежностью 0.95._

### 3. Задание:

_Известно, что рост футболистов в сборной распределен нормально
с дисперсией генеральной совокупности, равной 25 кв.см. Объем выборки равен 27,
среднее выборочное составляет 174.2. Найдите доверительный интервал для математического
ожидания с надежностью 0.95._

### 4. Задание:

_Выберите тему для проектной работы по курсу Теории вероятностей и математической статистики
и напишите ее в комментарии к Практическому заданию.
Предлагаем взять данные с сайта imdb.com_